In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
datadir = '/home/mei/nas/docker/dataset/EICU/eicu-collaborative-research-database-2.0/'
porcesseddir = '/home/mei/nas/docker/processedData/'

In [3]:
def round_up(x, base=5):
    return base * round(x/base)

In [5]:
df = pd.read_csv(datadir + 'medication.csv.gz', compression='gzip',dtype = {'loadingdose': 'str'})

In [6]:
df.head()

,medicationid,patientunitstayid,drugorderoffset,drugstartoffset,drugivadmixture,drugordercancelled,drugname,drughiclseqno,dosage,routeadmin,frequency,loadingdose,prn,drugstopoffset,gtc
0,7426715,141168,309,666,No,No,METOPROLOL TARTRATE 25 MG PO TABS,2102.0,25 3,PO,Q12H SCH,NaN,No,1826,0
1,9643232,141168,1847,1832,No,No,3 ML - IPRATROPIUM-ALBUTEROL 0.5-2.5 (3) MG/...,NaN,3 1,NEBULIZATION,Q4H Resp PRN,NaN,Yes,2047,0
2,10270090,141168,296,1386,No,No,ASPIRIN EC 81 MG PO TBEC,1820.0,81 3,PO,Daily,NaN,No,2390,0
3,9496768,141168,2048,2029,No,No,3 ML - IPRATROPIUM-ALBUTEROL 0.5-2.5 (3) MG/...,NaN,3 1,NEBULIZATION,Q4H Resp PRN,NaN,Yes,2390,0
4,11259680,141168,117,246,No,No,ENOXAPARIN SODIUM 40 MG/0.4ML SC SOLN,NaN,40 3,SC,Daily,NaN,No,1721,0


In [7]:
df_patient = pd.read_csv(porcesseddir + 'selected_patients.csv')

In [10]:
df_selected_addissionDrug = pd.merge(df_patient, df, on='patientunitstayid')
df_selected_addissionDrug.head()

,patientunitstayid,hospitaladmitoffset,gender,age,apacheadmissiondx,unitadmitsource,admissionweight,dischargeweight,unitdischargelocation,unitdischargestatus,...,drugordercancelled,drugname,drughiclseqno,dosage,routeadmin,frequency,loadingdose,prn,drugstopoffset,gtc
0,141168,0,Female,70,"Rhythm disturbance (atrial, supraventricular)",Direct Admit,84.3,85.8,Death,Expired,...,No,METOPROLOL TARTRATE 25 MG PO TABS,2102.0,25 3,PO,Q12H SCH,NaN,No,1826,0
1,141168,0,Female,70,"Rhythm disturbance (atrial, supraventricular)",Direct Admit,84.3,85.8,Death,Expired,...,No,3 ML - IPRATROPIUM-ALBUTEROL 0.5-2.5 (3) MG/...,NaN,3 1,NEBULIZATION,Q4H Resp PRN,NaN,Yes,2047,0
2,141168,0,Female,70,"Rhythm disturbance (atrial, supraventricular)",Direct Admit,84.3,85.8,Death,Expired,...,No,ASPIRIN EC 81 MG PO TBEC,1820.0,81 3,PO,Daily,NaN,No,2390,0
3,141168,0,Female,70,"Rhythm disturbance (atrial, supraventricular)",Direct Admit,84.3,85.8,Death,Expired,...,No,3 ML - IPRATROPIUM-ALBUTEROL 0.5-2.5 (3) MG/...,NaN,3 1,NEBULIZATION,Q4H Resp PRN,NaN,Yes,2390,0
4,141168,0,Female,70,"Rhythm disturbance (atrial, supraventricular)",Direct Admit,84.3,85.8,Death,Expired,...,No,ENOXAPARIN SODIUM 40 MG/0.4ML SC SOLN,NaN,40 3,SC,Daily,NaN,No,1721,0


In [11]:
df = df_selected_addissionDrug.copy()
df = df.set_index(['patientunitstayid', 'drugstartoffset'])
df = df.drop(df.index[np.where(df.index.get_level_values('drugstartoffset') < 0)])
df = df.drop(columns = ['medicationid', 'drugorderoffset'])
df.sort_index(level = ['patientunitstayid', 'drugstartoffset'], inplace = True)
# Resample every 5 mins
df.rename(round_up, level = 'drugstartoffset', inplace = True)
# Reorder columns
cols = ['drugstopoffset', 'drugname', 'drughiclseqno', 'gtc', 'dosage', 'loadingdose', 'frequency', 'drugordercancelled', 'drugivadmixture',  'prn', 'routeadmin','gender','age','apacheadmissiondx','unitadmitsource','admissionweight','dischargeweight','unitdischargelocation','unitdischargestatus']
df = df[cols]
# Lowercase name
df['name'] = df['drugname'].str.lower()

In [12]:
df.head()

drugstopoffset  \
patientunitstayid drugstartoffset                   
141168            50                         2050   
                  125                        1466   
                  125                         152   
                  245                        1721   
                  470                         513   

                                                                            drugname  \
patientunitstayid drugstartoffset                                                      
141168            50               3 ML  -  IPRATROPIUM-ALBUTEROL 0.5-2.5 (3) MG/...   
                  125              AMIODARONE 450 MG/250 ML D5W INFUSION (STD CON...   
                  125                                                            NaN   
                  245                          ENOXAPARIN SODIUM 40 MG/0.4ML SC SOLN   
                  470                       500 ML  -  SODIUM CHLORIDE 0.9 % IV SOLN   

                                   drughiclseqno  gtc dosage loadingdose  \
patientunitstayid drugstartoffset                                          
141168            50                         NaN    0    3 1         NaN   
                  125                    17793.0   38    NaN         NaN   
                  125                    17793.0   38  150 3         NaN   
                  245                        NaN    0   40 3         NaN   
                  470                     8255.0   59  500 1         NaN   

                                      frequency drugordercancelled  \
patientunitstayid drugstartoffset                                    
141168            50               4x Daily PRN                 No   
                  125                Continuous                 No   
                  125                      Once                 No   
                  245                     Daily                 No   
                  470                      Once                 No   

                                  drugivadmixture  prn    routeadmin  gender  \
patientunitstayid drugstartoffset                                              
141168            50                           No  Yes  NEBULIZATION  Female   
                  125                          No   No            IV  Female   
                  125                          No   No            IV  Female   
                  245                          No   No            SC  Female   
                  470                          No   No            IV  Female   

                                  age  \
patientunitstayid drugstartoffset       
141168            50               70   
                  125              70   
                  125              70   
                  245              70   
                  470              70   

                                                               apacheadmissiondx  \
patientunitstayid drugstartoffset                                                  
141168            50               Rhythm disturbance (atrial, supraventricular)   
                  125              Rhythm disturbance (atrial, supraventricular)   
                  125              Rhythm disturbance (atrial, supraventricular)   
                  245              Rhythm disturbance (atrial, supraventricular)   
                  470              Rhythm disturbance (atrial, supraventricular)   

                                  unitadmitsource  admissionweight  \
patientunitstayid drugstartoffset                                    
141168            50                 Direct Admit             84.3   
                  125                Direct Admit             84.3   
                  125                Direct Admit             84.3   
                  245                Direct Admit             84.3   
                  470                Direct Admit             84.3   

                                   dischargeweight unitdischargelocation  \
patientunitstayid

In [14]:
df.to_csv(porcesseddir + 'selected_medication.csv',  index=True)
print("Data exported successfully to 'selected_medication.csv'.")

Data exported successfully to 'selected_medication.csv'.


In [15]:
selected_med_p=len(list(df.index.get_level_values('patientunitstayid').unique()))
selected_med_x=len(df)
print("There are {} unique patientunitstayid in the selected patients with the total {} records.".format(selected_med_p,selected_med_x))

There are 83611 unique patientunitstayid in the selected patients with the total 2773930 records.


In [13]:
freq = pd.DataFrame()
freq['noAnnotations'] = df['name'].value_counts() 
freq['%'] = df['name'].value_counts() / len(df.index) * 100
freq

,noAnnotations,%
name,,
potassium chloride,22532,0.812277
acetaminophen,17421,0.628026
furosemide,15247,0.549653
sodium chloride 0.9%,14794,0.533323
novolog,14078,0.507511
...,...,...
mineral oil rect enem,78,0.002812
simethicone 80 mg oral chew tab,75,0.002704
sennosides-docusate sodium 8.6-50 mg oral tab,69,0.002487


In [16]:
df1 = df.copy()
df1 = df1.reset_index()
df1 = df1.groupby(["patientunitstayid", "name"])['drugstopoffset'].count().reset_index(name="count")
df1 = df1.sort_values(['count'],ascending=False)

df1

,patientunitstayid,name,count
365310,980146,nacl 0.9% mbp,262
365311,980146,nafcillin,261
344677,970196,propofol,148
330601,963353,propofol,114
341083,968569,nacl 0.9% mbp,110
...,...,...,...
72583,192947,mupirocin 2 % oint 1 g syringe,1
569437,1611866,haloperidol,1
569438,1611866,ipratropium-albuterol,1
247190,684169,magnesium sulfate 2 gm/50 ml ivpb,1


In [17]:
df2 = df1.groupby(['patientunitstayid']).apply(lambda x: x.sort_values(['count'], ascending=False).drop('patientunitstayid', axis=1))
df2 = df2.reset_index().drop(columns = 'level_1').set_index(['patientunitstayid', 'name'])
df2

/tmp/ipykernel_1549471/589263922.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df2 = df1.groupby(['patientunitstayid']).apply(lambda x: x.sort_values(['count'], ascending=False).drop('patientunitstayid', axis=1))


count
patientunitstayid name                                                     
141168            3 ml  -  ipratropium-albuterol 0.5-2.5 (3) mg/3...      4
                  1000 ml  -  sodium chloride 0.9 % iv soln               3
                  500 ml  -  sodium chloride 0.9 % iv soln                2
                  473 ml  -  chlorhexidine gluconate 0.12 % mt soln       2
                  2 ml vial : midazolam hcl 2 mg/2ml ij soln              2
...                                                                     ...
3353263           hydromorphone hcl                                       1
                  morphine 2mg/ml                                         1
                  naloxone hcl                                            1
                  ondansetron hcl                                         1
                  oxycodone/apap 5mg/325mg                                1

[1083808 rows x 1 columns]

In [1]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import psycopg2
import getpass
import pdvega
import seaborn as sns
# for configuring connection 
from configobj import ConfigObj
import os

%matplotlib inline

In [2]:
# Create a database connection using settings from config file
config='../db/config.ini'

# connection info
conn_info = dict()
if os.path.isfile(config):
    config = ConfigObj(config)
    conn_info["sqluser"] = config['username']
    conn_info["sqlpass"] = config['password']
    conn_info["sqlhost"] = config['host']
    conn_info["sqlport"] = config['port']
    conn_info["dbname"] = config['dbname']
    conn_info["schema_name"] = config['schema_name']
else:
    conn_info["sqluser"] = 'postgres'
    conn_info["sqlpass"] = ''
    conn_info["sqlhost"] = 'localhost'
    conn_info["sqlport"] = 5432
    conn_info["dbname"] = 'eicu'
    conn_info["schema_name"] = 'public,eicu_crd'
    
# Connect to the eICU database
print('Database: {}'.format(conn_info['dbname']))
print('Username: {}'.format(conn_info["sqluser"]))
if conn_info["sqlpass"] == '':
    # try connecting without password, i.e. peer or OS authentication
    try:
        if (conn_info["sqlhost"] == 'localhost') & (conn_info["sqlport"]=='5432'):
            con = psycopg2.connect(dbname=conn_info["dbname"],
                                   user=conn_info["sqluser"])            
        else:
            con = psycopg2.connect(dbname=conn_info["dbname"],
                                   host=conn_info["sqlhost"],
                                   port=conn_info["sqlport"],
                                   user=conn_info["sqluser"])
    except:
        conn_info["sqlpass"] = getpass.getpass('Password: ')

        con = psycopg2.connect(dbname=conn_info["dbname"],
                               host=conn_info["sqlhost"],
                               port=conn_info["sqlport"],
                               user=conn_info["sqluser"],
                               password=conn_info["sqlpass"])
query_schema = 'set search_path to ' + conn_info['schema_name'] + ';'

Database: eicu
Username: postgres


In [3]:
from sqlalchemy import create_engine
con= create_engine('postgresql://eicu@localhost:5432/eicu')

In [4]:
def round_up(x, base=5):
    return base * round(x/base)